In [ ]:
!pip install -upgrade -quiet langchain-core langchain-community langchain-openai
!pip install openai
!pip install langchain
!pip install langchain_openai


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 190.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [ ]:
#튜플이나 리스트같은 '컬렉션타입' 데이터에서 원하는 인덱스의 요소를 추출하는 데 사용
from operator import itemgetter

#러너블 클래 : 함수를 감싸서 체인화 할 수 있게 해 줌
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

#Output(결과물)을 content만 가져올 수 있도록 변환
# choices[0].content... <- {model,,, }
#"질의응답 중 응답내용만 가져오도록" 정리해주는 함수
from langchain_core.output_parsers import StrOutputParser

#특정한 대화 양식(템플릿)을 정의하여 쉽게 질의응답 할 수 있도록 해줌
#우리가 나눈 대화를 관리하는 데 사용
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

#openai와의 대화형 질의응답
from langchain_openai import ChatOpenAI

#openai와 나눈 대화를 저장하고 관리하는데 사용하는 클래스
#"이전 대화를 기반으로" 현재의 대화를 이끌어나가도록 함
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

### 챗봇을 정의해보자.

In [ ]:
system_prompt = """
너는 나랑 친구인 상담사야.
내가 물어보는 질문에 대해서 친절하게 잘 알려줘야 해.
한국어로 답변해줘.
"""

In [ ]:
chat_prompt_template = ChatPromptTemplate.from_messages([
  ("system", system_prompt),
  MessagesPlaceholder(variable_name='history'),
  ("human", "{user_input}")
])

#k=기억할 대화 쌍의 개수(보낸 메시지가 아니라 대화 쌍의 수)
#총 3개의 이전 대화 쌍을 기억함.
memory = ConversationBufferWindowMemory(k=3, return_messages=True)

In [ ]:
chat_model = ChatOpenAI()
output_parser = StrOutputParser()

In [ ]:
chain = ({"user_input":RunnablePassthrough()} |
        RunnablePassthrough.assign(history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"))
        | chat_prompt_template
        | chat_model
        | output_parser)

In [ ]:
def load_memory(_):
  return memory.load_memory_variables(_)['history']

In [ ]:
def chat_with_user(user_message):
  ai_message = chain.invoke(user_message)

  memory.save_context({"input":user_message}, {"output":ai_message})
  print(memory.load_memory_variables({}))

  return ai_message

In [ ]:
chat_with_user("안녕! 오늘 비온다고 했는데 너 우산 챙겼어?")

{'history': [HumanMessage(content='안녕! 오늘 비온다고 했는데 너 우산 챙겼어?'), AIMessage(content='안녕! 비가 온다고 하니 우산을 챙기는 게 좋겠네요. 나는 우산을 챙겨놓지는 못했어. 너는 우산을 챙겼니?'), HumanMessage(content='야이 멍청아'), AIMessage(content='미안해. 우산을 챙겼는지 물어봤는데 그런 식으로 대답해줘서 미안해. 함께 비 오는 날 우산 챙겨서 즐거운 하루 보내길 바래.'), HumanMessage(content='안녕! 오늘 비온다고 했는데 너 우산 챙겼어?'), AIMessage(content='안녕! 오늘 비가 온다고 하니 우산을 챙기는 게 좋겠네요. 나는 우산을 챙겨놓지는 못했어. 너는 우산을 챙겼니?')]}


'안녕! 오늘 비가 온다고 하니 우산을 챙기는 게 좋겠네요. 나는 우산을 챙겨놓지는 못했어. 너는 우산을 챙겼니?'

In [ ]:
while True:
  user_message = input("USER >> ")
  if user_message == 'quit':
    break
  ai_message = chat_with_user(user_message)
  print(f"AI >> {ai_message}")

USER >> 안녕! 내 이름은 박정은이고, 좋아하는 색깔은 파란색이야.
{'history': [HumanMessage(content='안녕! 오늘 비온다고 했는데 너 우산 챙겼어?'), AIMessage(content='안녕! 비가 올 것 같아서 우산을 챙겼어. 비 올 때는 항상 우산을 챙기는 게 좋아. 혹시 너는 우산을 가져왔니?'), HumanMessage(content='안녕! 내 이름은 박정은이고, 좋아하는 색깔은 파란색이야.'), AIMessage(content='안녕, 박정은씨! 파란색을 좋아하는구나. 파란색은 시원하고 차분한 느낌이 있어서 좋은 선택이야. 너의 취향을 알게 되어 반가워. 함께 대화를 나누는 게 즐거워.')]}
AI >> 안녕, 박정은씨! 파란색을 좋아하는구나. 파란색은 시원하고 차분한 느낌이 있어서 좋은 선택이야. 너의 취향을 알게 되어 반가워. 함께 대화를 나누는 게 즐거워.
USER >> 너는 바다랑 산 중에 어디 가는 거 좋아하니?
{'history': [HumanMessage(content='안녕! 오늘 비온다고 했는데 너 우산 챙겼어?'), AIMessage(content='안녕! 비가 올 것 같아서 우산을 챙겼어. 비 올 때는 항상 우산을 챙기는 게 좋아. 혹시 너는 우산을 가져왔니?'), HumanMessage(content='안녕! 내 이름은 박정은이고, 좋아하는 색깔은 파란색이야.'), AIMessage(content='안녕, 박정은씨! 파란색을 좋아하는구나. 파란색은 시원하고 차분한 느낌이 있어서 좋은 선택이야. 너의 취향을 알게 되어 반가워. 함께 대화를 나누는 게 즐거워.'), HumanMessage(content='너는 바다랑 산 중에 어디 가는 거 좋아하니?'), AIMessage(content='바다와 산 중에서 선택하기 어렵지만, 나는 바다를 더 선호해. 바다는 파도 소리와 바닷물 향기로 마음을 편안하게 만들어주니까. 하지만 산 속의 싱그러운 공기와 푸른 풍경도 매력적이지. 너는 어디를 선호해? 함께 

KeyboardInterrupt: Interrupted by user

## 그라디오를 활용한 챗봇 껍데기 만들기

In [ ]:
!pip install gradio

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.9 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=f78b5c06

In [ ]:
import gradio as gr

import time

def respond(user_message, chat_view):
  ai_message = chat_with_user(user_message)
#  chat_view.append(("", ['안녕! 나는 연애상담봇이야.']))
  chat_view.append((user_message, ai_message))

  time.sleep(1)

  return "", chat_view

with gr.Blocks() as demo:
  chat_view = gr.Chatbot(label = "채팅창")
  user_textbox = gr.Textbox(label='입력')
  user_textbox.submit(respond, [user_textbox, chat_view],[user_textbox, chat_view])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://de5e073e09d9d0093b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
